In [ ]:
import os
if 'google.colab' in str(get_ipython()) and not os.path.exists('/content/TGCN-PyTorch'):
    !git clone https://github.com/mamintoosi-papers-codes/TGCN-PyTorch.git
    !pip install -q torchmetrics
    %cd TGCN-PyTorch

Cloning into 'TGCN-PyTorch'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 265 (delta 163), reused 209 (delta 110), pack-reused 0 (from 0)
Receiving objects: 100% (265/265), 7.81 MiB | 8.59 MiB/s, done.
Resolving deltas: 100% (163/163), done.
/content/TGCN-PyTorch


In [4]:
%%time

datasets = ['los', 'sz'] # los=losloop, sz=shenzhen
pred_list = [1, 2, 3, 4]

for  dataset in datasets:
    for pre_len in pred_list:
        %run main.py --config configs/tgcn-{dataset}-pre_len{pre_len}.yaml
        %run main.py --config configs/tgcn-{dataset}-gsl-pre_len{pre_len}.yaml
        %run main.py --config configs/tgcn-{dataset}-gsl-adj-pre_len{pre_len}.yaml

[2025-03-04 20:07:51,948 INFO]Loaded config from configs/tgcn-los-pre_len1.yaml: {'fit': {'trainer': {'max_epochs': 50, 'accelerator': 'cuda', 'devices': 1}, 'data': {'dataset_name': 'losloop', 'batch_size': 64, 'seq_len': 12, 'pre_len': 1}, 'model': {'model': {'class_path': 'models.TGCN', 'init_args': {'hidden_dim': 64, 'use_gsl': 0}}, 'learning_rate': 0.001, 'weight_decay': 0, 'loss': 'mse_with_regularizer'}}}
[2025-03-04 20:07:52,304 INFO]Using device: cuda
[2025-03-04 20:07:52,606 INFO]Starting training for 50 epochs...
[2025-03-04 20:07:54,393 INFO][Epoch 1/50] Train Loss: 2284.682479, Val Loss: 6375080.000000, RMSE: 12.551171, MAE: 8.776714, Accuracy: 0.7864, R2: 0.3256, Expl.Var: 0.3963
[2025-03-04 20:08:02,256 INFO][Epoch 11/50] Train Loss: 113.115096, Val Loss: 3793821.250000, RMSE: 9.682330, MAE: 6.945940, Accuracy: 0.8352, R2: 0.5116, Expl.Var: 0.5117
[2025-03-04 20:08:10,251 INFO][Epoch 21/50] Train Loss: 86.276583, Val Loss: 2802929.750000, RMSE: 8.322380, MAE: 5.861984, A

CPU times: user 23min 4s, sys: 6.18 s, total: 23min 10s
Wall time: 23min 32s


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil


# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]
datasets = ['losloop', 'shenzhen']

# Define a color palette for each method
colors = plt.cm.tab10.colors  # Using a predefined color palette (10 distinct colors)
method_colors = {
    "T-GCN": colors[0],           # T-GCN (Default)
    "T-GCN (GSL Only)": colors[1],  # T-GCN (GSL Only)
    "T-GCN (GSL + Adj)": colors[2], # T-GCN (GSL + Adj)
}

# Define line styles for each method
line_styles = {
    "T-GCN": "--",                # Dashed for T-GCN (Default)
    "T-GCN (GSL Only)": "-",       # Solid for T-GCN (GSL Only)
    "T-GCN (GSL + Adj)": ":",      # Dotted for T-GCN (GSL + Adj)
}

# Define markers for each method
markers = {
    "T-GCN": "o",                 # Circle for T-GCN (Default)
    "T-GCN (GSL Only)": "s",       # Square for T-GCN (GSL Only)
    "T-GCN (GSL + Adj)": "D",      # Diamond for T-GCN (GSL + Adj)
}

# Ensure the results directory exists
os.makedirs("results", exist_ok=True)

# Loop through each dataset
for dataset in datasets:
    # Loop through each measure
    for measure in measures:
        # Loop through each pre_len in pred_list
        for pre_len in pred_list:
            # Create a new figure for each pre_len and measure
            plt.figure(figsize=(12, 8))

            # Load the metrics for T-GCN (Default) for comparison
            metrics_file_tgcn = f"results/metrics_{dataset}_TGCN_seq12_pre{pre_len}_gsl0.csv"
            metrics_df_tgcn = pd.read_csv(metrics_file_tgcn)

            # Loop through each method
            for method, color in method_colors.items():
                # Map method names to their corresponding file suffixes
                if method == "T-GCN":
                    file_suffix = "gsl0"
                elif method == "T-GCN (GSL Only)":
                    file_suffix = "gsl1"
                elif method == "T-GCN (GSL + Adj)":
                    file_suffix = "gsl2"

                # Generate the file path dynamically based on pre_len and method
                metrics_file = f"results/metrics_{dataset}_TGCN_seq12_pre{pre_len}_{file_suffix}.csv"

                # Load the metrics CSV file
                metrics_df = pd.read_csv(metrics_file)

                # Plot the current measure for the current method
                plt.plot(
                    metrics_df["Epoch"],
                    metrics_df[measure],
                    linestyle=line_styles[method],
                    marker=markers[method],
                    color=color,
                    label=f"{method} (pre_len={pre_len})",
                )

                # Highlight the first epoch where the current method is better than T-GCN (Default)
                if method != "T-GCN":  # Skip comparison for T-GCN (Default)
                    better_epoch = None
                    for epoch in range(len(metrics_df)):
                        tgcn_value = metrics_df_tgcn.loc[epoch, measure]
                        current_value = metrics_df.loc[epoch, measure]

                        # Check if the current method is better than T-GCN (Default)
                        if measure in ["RMSE", "MAE"]:
                            if current_value < tgcn_value:  # Lower is better
                                better_epoch = epoch + 1  # Epochs are 1-indexed
                                break
                        else:
                            if current_value > tgcn_value:  # Higher is better
                                better_epoch = epoch + 1
                                break

                    # Highlight the first epoch where the current method is better
                    if better_epoch is not None:
                        plt.axvline(
                            x=better_epoch,
                            color=color,
                            linestyle=":",
                            alpha=0.5,
                            label=f"{method} better at epoch {better_epoch} (pre_len={pre_len})",
                        )

            # Add labels, title, legend, and grid
            plt.xlabel("Epoch")
            plt.ylabel(measure)
            plt.title(f"{measure} over Epochs for {dataset} (pre_len={pre_len})")
            plt.legend()
            plt.grid(True)

            # Save the figure as a PNG file
            filename = f"results/{dataset}_{measure}_pre{pre_len}.png"
            plt.savefig(filename, bbox_inches="tight", dpi=300)
            plt.close()  # Close the figure to free up memory


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
import pandas as pd
import os

# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]
datasets = ['losloop', 'shenzhen']

# Ensure the results directory exists
os.makedirs("results", exist_ok=True)

# Loop through each dataset
for dataset in datasets:
    # Initialize a dictionary to store the results of the last epoch
    results = {}

    # Loop through each pre_len and measure to collect the last epoch results
    for pre_len in pred_list:
        # Generate the file paths dynamically based on pre_len and method
        metrics_file_gsl0 = f"results/metrics_{dataset}_TGCN_seq12_pre{pre_len}_gsl0.csv"
        metrics_file_gsl1 = f"results/metrics_{dataset}_TGCN_seq12_pre{pre_len}_gsl1.csv"
        metrics_file_gsl2 = f"results/metrics_{dataset}_TGCN_seq12_pre{pre_len}_gsl2.csv"

        # Load the metrics CSV files
        metrics_df_gsl0 = pd.read_csv(metrics_file_gsl0)
        metrics_df_gsl1 = pd.read_csv(metrics_file_gsl1)
        metrics_df_gsl2 = pd.read_csv(metrics_file_gsl2)

        # Get the last epoch results for all methods
        last_epoch_gsl0 = metrics_df_gsl0.iloc[-1]  # Last row for gsl0 (T-GCN)
        last_epoch_gsl1 = metrics_df_gsl1.iloc[-1]  # Last row for gsl1 (T-GCN GSL Only)
        last_epoch_gsl2 = metrics_df_gsl2.iloc[-1]  # Last row for gsl2 (T-GCN GSL + Adj)

        # Store the results in the dictionary with pre_len as part of the key
        results[f"T-GCN (pre_len={pre_len})"] = last_epoch_gsl0[measures]
        results[f"T-GCN (GSL Only) (pre_len={pre_len})"] = last_epoch_gsl1[measures]
        results[f"T-GCN (GSL + Adj) (pre_len={pre_len})"] = last_epoch_gsl2[measures]

    # Convert the results dictionary to a DataFrame
    results_df = pd.DataFrame(results).T

    # Add a column for pre_len to results_df
    results_df["pre\\_len"] = [pre_len for pre_len in pred_list for _ in range(3)]

    # Function to highlight the winner for each pre_len and measure
    def highlight_winner(df):
        highlighted_df = df.copy()
        for pre_len in pred_list:
            for measure in measures:
                # Get the values for the three methods for the current pre_len
                tgcn_value = df.loc[f"T-GCN (pre_len={pre_len})", measure]
                gsl_only_value = df.loc[f"T-GCN (GSL Only) (pre_len={pre_len})", measure]
                gsl_adj_value = df.loc[f"T-GCN (GSL + Adj) (pre_len={pre_len})", measure]

                # Determine the winner based on the measure
                if measure in ["RMSE", "MAE"]:
                    winner_value = min(tgcn_value, gsl_only_value, gsl_adj_value)  # Lower is better
                else:
                    winner_value = max(tgcn_value, gsl_only_value, gsl_adj_value)  # Higher is better

                # Highlight the winner
                if tgcn_value == winner_value:
                    highlighted_df.loc[f"T-GCN (pre_len={pre_len})", measure] = f"\\textbf{{{tgcn_value:.4f}}}"
                else:
                    highlighted_df.loc[f"T-GCN (pre_len={pre_len})", measure] = f"{tgcn_value:.4f}"

                if gsl_only_value == winner_value:
                    highlighted_df.loc[f"T-GCN (GSL Only) (pre_len={pre_len})", measure] = f"\\textbf{{{gsl_only_value:.4f}}}"
                else:
                    highlighted_df.loc[f"T-GCN (GSL Only) (pre_len={pre_len})", measure] = f"{gsl_only_value:.4f}"

                if gsl_adj_value == winner_value:
                    highlighted_df.loc[f"T-GCN (GSL + Adj) (pre_len={pre_len})", measure] = f"\\textbf{{{gsl_adj_value:.4f}}}"
                else:
                    highlighted_df.loc[f"T-GCN (GSL + Adj) (pre_len={pre_len})", measure] = f"{gsl_adj_value:.4f}"
        return highlighted_df

    # Apply the highlight function to the results DataFrame
    highlighted_results = highlight_winner(results_df)

    # Remove (pre_len={pre_len}) from method names
    highlighted_results.index = highlighted_results.index.str.replace(r" \(pre_len=\d+\)", "", regex=True)

    # Add a column for method names
    highlighted_results.insert(0, "Method", highlighted_results.index)

    # Reorder columns to make pre_len the first column and method-name the second column
    highlighted_results = highlighted_results[["pre\\_len", "Method"] + measures]

    # Generate the LaTeX table
    latex_table = highlighted_results.to_latex(
        escape=False,
        column_format="cl" + "c" * len(measures),  # Updated column format
        multicolumn_format="c",
        index=False  # Do not include the index in the LaTeX table
    )

    # Save the LaTeX table to a file
    table_filename = f"results/{dataset}_results_table.tex"
    with open(table_filename, "w") as f:
        f.write(latex_table)

    # Print the LaTeX table
    print(f"LaTeX table for {dataset}:")
    print(latex_table)

LaTeX table for losloop:
\begin{tabular}{clcccc}
\toprule
 pre\_len &            Method &            RMSE &             MAE &        Accuracy &              R2 \\
\midrule
        1 &             T-GCN &          6.8539 &          4.7620 &          0.8833 &          0.7552 \\
        1 &  T-GCN (GSL Only) & \textbf{5.7027} & \textbf{3.7524} & \textbf{0.9029} & \textbf{0.8305} \\
        1 & T-GCN (GSL + Adj) &          6.4339 &          4.4765 &          0.8905 &          0.7843 \\
        2 &             T-GCN &          7.3917 &          5.0901 &          0.8742 &          0.7156 \\
        2 &  T-GCN (GSL Only) & \textbf{6.4694} & \textbf{4.2092} & \textbf{0.8899} & \textbf{0.7823} \\
        2 & T-GCN (GSL + Adj) &          6.9092 &          4.8106 &          0.8824 &          0.7516 \\
        3 &             T-GCN & \textbf{7.2658} &          4.9950 & \textbf{0.8763} & \textbf{0.7256} \\
        3 &  T-GCN (GSL Only) &          7.2749 & \textbf{4.6912} &          0.8762 &        

In [ ]:
if 'google.colab' in str(get_ipython()): 
    from google.colab import files
    # Zip the results folder
    shutil.make_archive('results', 'zip', 'results')
    # Download the zipped file
    files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>